In [ ]:
import hvplot.pandas
import pandas as pd
import requests
from api_keys import geoapify_key

In [ ]:
city_data_df = pd.read_csv("data/output_data/cities.csv")

city_data_df.tail()

In [27]:
import hvplot.pandas
import numpy as np

scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    size='Humidity',
    hover_cols=['City'],
    title='City Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    frame_width=700,
    frame_height=500,
    color=np.random.choice(['red', 'green', 'blue', 'orange', 'purple'], size=len(city_data_df)),
    cmap=None,  
    colorbar=False,  
    tiles='CartoLight', 
)

scatter_plot


:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [Lng]   (Lat,_color,Humidity,City)

In [ ]:
temp_df = city_data_df[(city_data_df['Max Temp'] <= 60) & 
                       (city_data_df['Max Temp'] >= 40) & 
                       (city_data_df['Wind Speed'] < 4.5) &
                       (city_data_df['Cloudiness'] == 0)] 


print(temp_df)

In [ ]:
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df["Hotel Name"] = ""
print(hotel_df)


In [24]:
import requests
import pandas as pd

hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = ""

radius = 10000 

print("Starting hotel search")

for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    base_url = "https://api.geoapify.com/v2/places"
    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{longitude},{latitude},{radius}',
        'bias': f'proximity:{longitude},{latitude}',
        'limit': 1,
        'apiKey': geoapify_key  
    }

    response = requests.get(base_url, params=params)
    name_address = response.json()

    if name_address.get('features') and name_address['features']:
        hotel_name = name_address['features'][0]['properties'].get('name', 'No hotel name available')
    else:
        hotel_name = "No hotel found"

    hotel_df.loc[index, 'Hotel Name'] = hotel_name
    print(f"{row['City']} - nearest hotel: {hotel_name}")

print("Hotel search completed")
hotel_df.head

Starting hotel search
Hotel search completed


<bound method NDFrame.head of                   City Country      Lat       Lng  Humidity  \
0                 Lebu      CL -37.6167  -73.6500        87   
1    Port-Aux-Francais      TF -49.3500   70.2167        71   
2             Vallenar      CL -28.5708  -70.7581        75   
3       Puerto Natales      CL -51.7236  -72.4875        92   
4                 Bodo      NO  67.2800   14.4050        89   
..                 ...     ...      ...       ...       ...   
538              Wuxue      CN  29.8506  115.5525        82   
539              Agdam      AZ  39.9910   46.9274        89   
540              Taroa      MH   8.7058  171.2275        93   
541              Heihe      CN  50.2441  127.4902        74   
542         Shitanjing      CN  39.2342  106.3439        18   

                   Hotel Name  
0              No hotel found  
1                     Keravel  
2           Hotel del Marqués  
3    Dorotea Patagonia Hostel  
4                Zefyr hotell  
..                   

In [39]:
import hvplot.pandas
import numpy as np

# Number of points in your DataFrame
num_points = len(hotel_df)

# Generate colors using the 'viridis' colormap
colors = plt.cm.viridis(np.linspace(0, 1, num_points))
colors = np.random.choice([ '#f67280',  # Sunset orange
    '#c06c84',
    '#6c5b7b',  
    '#355c7d', 
    '#99b898', 
    '#2a9d8f',  
    '#e9c46a', 
    '#f4a261',  
    '#e76f51',  
    '#264653' ], size=len(city_data_df))

# Create a scatter plot using hvplot with a basemap
scatter_plot = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    size='Humidity',
    hover_cols=['City', 'Country', 'Hotel Name'],  # Include 'Country' and 'Hotel Name' in hover tool
    title='City Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    frame_width=700,
    frame_height=500,
    color=colors, 
    cmap=None,  
    colorbar=False,  
    tiles='CartoLight',
)

scatter_plot


:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [Lng]   (Lat,_color,Humidity,City,Country,Hotel Name)